In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
np.random.seed(132)
from functools import lru_cache

import sys

CODE_PATH = '../code'

sys.path.append(CODE_PATH)
import functions


from sklearn.datasets import load_iris, load_boston, load_breast_cancer, load_wine, load_digits
from scipy.optimize import minimize
from tqdm import tqdm
%pylab inline

Populating the interactive namespace from numpy and matplotlib


бостон - градиент уходит в 0  
ирис - спустя пару итераций в 0

In [3]:
# dataset = load_breast_cancer()
dataset = load_digits()
df = pd.DataFrame(dataset['data'])
target = dataset['target']
# df = (df - df.mean())/(df.max() - df.min())
# df0 = df.copy()
print(df.shape)
print(target)
df.head()

# train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.values, target, test_size=0.33, random_state=42)

(1797, 64)
[0 1 2 ... 8 9 8]


# Формируем дихотомическую матрицу

In [4]:
# num_classes
l = np.unique(target).size
N = 20 # кол-во дихотомий

In [5]:
# формируем дих матрицу l*N
code_matrix = np.zeros((l, N))
def add_random_dich(l=10, code_matrix=None):
    if code_matrix is None:
        # матрица пуста
        dich = np.random.randint(0, 2, l)
        while np.unique(dich).size == 1:
            dich = np.random.randint(0, 2, l)
        return dich.reshape((-1, 1))
    # матрица непуста
    dich = np.random.randint(0, 2, l)
    def does_dich_exist(dich, code_matrix):
        diff = (cm == dich).sum(axis=0)
        if diff.max() == l or diff.min() == 0:
            return True
        return False
    while np.unique(dich).size == 1 and not does_dich_exist(dich, code_matrix):
        dich = np.random.randint(0, 2, l)
#     print(code_matrix.shape, dich.shape)
    return np.hstack([code_matrix, dich.reshape((-1, 1))])

In [6]:
code_matrix = None
for i in tqdm(range(N), desc='Adding dich'):
    code_matrix = add_random_dich(l, code_matrix)

Adding dich: 100%|██████████| 20/20 [00:00<00:00, 12131.03it/s]


# Поиск дихотомии через оптимум

In [ ]:
# начальная точка максимально удаленная от линейной оболочки дихотомий


# Обучаем классификаторы

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

BaseClassifier = LogisticRegression
dich_classifiers = []
for i in tqdm(range(N), desc='Training dich classifiers'):
    clf = BaseClassifier()
    X = X_train
    y_classes = code_matrix.T[i]
    y = np.array([y_classes[i] for i in y_train])
    clf.fit(X, y)
    y_pred = clf.predict(X_test)
    y_true = np.array([y_classes[i] for i in y_test])
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    true_mask = (y_pred == y_true)
    confusion_list = np.array([np.sum(true_mask*(y_test==i))/np.sum(y_test==i) for i in range(l)])
    dich_classifiers.append({'model': clf, 'accuracy': accuracy, 'f1': f1, 'confusion_list': confusion_list})

Training dich classifiers: 100%|██████████| 20/20 [00:00<00:00, 35.34it/s]


In [8]:
def predict_codeword(x, dich_classifiers):
    codeword = []
    for dich in dich_classifiers:
        clf = dich['model']
        codeword.append(clf.predict(x.reshape(1, -1)))
    return np.array(codeword).flatten()

def hamming(arr1, arr2, scores=None):
#     print(arr1, arr2, scores)
    if scores is None:
        return (arr1 != arr2).sum()
    return ((arr1 != arr2)*scores).sum() + ((arr1 == arr2)*(1-scores)).sum()
    
def predict_class(x, dich_classifiers, code_matrix, score_type=None, verbose=False):
    codeword = predict_codeword(x, dich_classifiers)
    if not score_type:
        hammings = np.array([hamming(codeword, class_code) for class_code in code_matrix])
    else:
        scores = np.array([d[score_type] for d in dich_classifiers])
        if score_type == 'confusion_list':
            # ПРОВЕРИТЬ ВЕРНО ЛИ ФОРМИРУЮТСЯ ОЦЕНКИ ТУТ
            hammings = np.array([hamming(codeword, class_code, scores.T[i]) \
                                 for i, class_code in enumerate(code_matrix)])
        else:
            hammings = np.array([hamming(codeword, class_code, scores) for class_code in code_matrix])
    if verbose:
        print(hammings)
    indices = np.where(hammings == hammings.min())
    return np.random.choice(indices[0])

# Сравнения весов (без отбора дихотомий)
Сравним разные классификации (хемминг, точность, ф-мера, обучающие данные)

In [9]:
preds = [predict_class(x, dich_classifiers, code_matrix) for x in X_test]
from sklearn.metrics import accuracy_score
accuracy_score(preds, y_test)

0.8956228956228957

In [10]:
preds = [predict_class(x, dich_classifiers, code_matrix, 'accuracy') for x in X_test]
from sklearn.metrics import accuracy_score
accuracy_score(preds, y_test)

0.9006734006734006

In [11]:
preds = [predict_class(x, dich_classifiers, code_matrix, 'f1') for x in X_test]
from sklearn.metrics import accuracy_score
accuracy_score(preds, y_test)

0.9057239057239057

In [12]:
preds = [predict_class(x, dich_classifiers, code_matrix, 'confusion_list') for x in X_test]
from sklearn.metrics import accuracy_score
accuracy_score(preds, y_test)

0.9040404040404041

# Оптимизация с произвольными метриками

In [54]:
from functions import get_weights_gap
get_weights_gap(code_matrix, dich_classifiers, 'accuracy')

Optimization terminated successfully.
         Current function value: -4.820034   
         Iterations: 16


array([ 0.00000000e+00,  0.00000000e+00,  2.88099258e+00,  3.28728640e+00,
        0.00000000e+00,  0.00000000e+00,  1.14130344e-15,  0.00000000e+00,
        0.00000000e+00,  4.11350277e+00,  0.00000000e+00,  8.88178420e-16,
        3.44943954e+00, -3.52897242e-16,  0.00000000e+00,  0.00000000e+00,
        2.98149232e+00,  0.00000000e+00,  0.00000000e+00,  3.28728640e+00])


# Конструируем модельную задачу

In [ ]:
np.random.seed(100)
cluster_objects = 100
num_clusters = 5

mean = np.zeros(2)
cov = np.eye(2) * 0.07
X = []

y = np.zeros(num_clusters**2*cluster_objects)

for i in np.linspace(0, num_clusters-1, num_clusters):
    for j in np.linspace(0, num_clusters-1, num_clusters):
        mean = np.array([i, j])
        X_cluster = np.random.multivariate_normal(mean, cov, cluster_objects)
        X.append(X_cluster)
X = np.vstack(X)
scatter(X[:,0], X[:,1], )

In [ ]:
# объединяем классы
classes_in_cluster = 5

order = np.array(range(num_clusters**2))
np.random.shuffle(order)
splits = np.split(order, 5)
for i, split in enumerate(splits):
    for item in split:
        y[item*cluster_objects:(item+1)*cluster_objects] = i

In [ ]:
print(y)
plot(y)

In [ ]:
figsize(8, 8)
scatter(X[:,0], X[:,1], c=y, )

In [ ]:
# модельная задача на 12 классов
colors = []
cs = []

# np.random.seed(100)
cluster_objects = 100
num_clusters = 5

mean = np.zeros(2)
cov = np.eye(2) * 0.07
X = []

y = np.zeros(num_clusters**2*cluster_objects)

for i in np.linspace(0, 4-1, 4):
    for j in np.linspace(0, 5-1, 5):
        c = np.random.randint(0, 12)
        mean = np.array([i, j])
        X_cluster = np.random.multivariate_normal(mean, cov, cluster_objects)
        X.append(X_cluster)
        colors += [c]*cluster_objects
        cs.append(c)
X = np.vstack(X)
print(cs)
scatter(X[:,0], X[:,1], c=colors)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.multioutput import ClassifierChain
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import jaccard_similarity_score
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import fetch_mldata

# Load a multi-label dataset
yeast = fetch_mldata('yeast')
X = yeast['data']
Y = yeast['target'].transpose().toarray()
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2,
                                                    random_state=0)

In [ ]:
yeast['target']

In [ ]:
df = pd.read_csv('../yeast.data.txt', sep=';', header=None)
X = df.values[:,1:-1]
y = pd.factorize(df[9])[0]
y

In [ ]:
from sklearn.manifold import TSNE


tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(X)

In [ ]:
scatter(tsne_results[:,0], tsne_results[:,1], c=y)
